# Capstone Project - The Battle of Neighborhoods (Week 5)
#### Week 2 Final Notebook

**_Opening a New Shopping Mall in Bangalore, India_**
- Build a dataframe of neighborhoods in Bangalore, India by web scraping the data from Wikipedia page
- Get the geographical coordinates of all the neighborhoods
- Obtain the venue data for the neighborhoods from the Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***
### 1. Import libraries

In [254]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder  #to get the latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Download data from Wikipedia page and convert it into a _pandas_ dataframe

In [255]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore").text

In [256]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [257]:
# create a list to store neighborhood data
neighborhoodList = []

In [258]:
# append the data into the list
for i in range(0,8):
    for row in soup.find_all("table", class_="wikitable sortable")[i].find_all("td"):
        neighborhoodList.append(row.text)
neighborhoodList = neighborhoodList[::3]
neighborhoodList = ([s.strip("\n") for s in neighborhoodList]) # remove \n from the string borders

neighborhoodList[0] = 'Catonment area, Bangalore'  # Clarifying to Geocoder so that it doesn't take the latitude and longitude of the Cantonment area in Delhi

neighborhoodList.append('Attibele')            # These are the additional areas in Bangalore Urban District that weren't properly
neighborhoodList.append('Chandrapura')         # mentioned in the Wikipedia page. 
neighborhoodList.append('Thavarekere')
neighborhoodList.append('Chikkabanavara')
neighborhoodList.append('Hesaraghatta')
neighborhoodList.append('Jigani')
neighborhoodList.append('Sarjapura')

In [259]:
# create a new DataFrame from the list
blr_df = pd.DataFrame({"Neighborhood": neighborhoodList})

blr_df.head(200)

,Neighborhood
0,"Catonment area, Bangalore"
1,Domlur
2,Indiranagar
3,Jeevanbheemanagar
4,Malleswaram
5,Pete area
6,Sadashivanagar
7,Seshadripuram
8,Shivajinagar
9,Ulsoor


In [260]:
# print the number of rows of the dataframe
blr_df.shape

(72, 1)

### 3. Getting the geographical coordinates of all neighborhoods

In [261]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [262]:
# call the function to get the coordinates, store in a new list using list comprehension
nbrhd_coordinates = [ get_latlng(neighborhood) for neighborhood in blr_df["Neighborhood"].tolist() ]

In [264]:
nbrhd_coordinates

[[12.966180000000065, 77.58690000000007],
 [12.943290000000047, 77.65602000000007],
 [12.973940000000027, 77.64390000000003],
 [12.96601000000004, 77.65767000000005],
 [13.006322454623541, 77.56841583882036],
 [12.966180000000065, 77.58690000000007],
 [13.014830000000075, 77.57771000000008],
 [12.993550000000027, 77.57988000000006],
 [12.98720000000003, 77.60401000000007],
 [12.989080000000058, 77.62795000000006],
 [12.990730000000042, 77.58861000000007],
 [12.927350000000047, 77.67185000000006],
 [12.979018801404825, 77.65613753365203],
 [12.99198000000007, 77.71506000000005],
 [13.000390000000039, 77.68368000000004],
 [12.994090000000028, 77.66633000000007],
 [12.954660000000047, 77.70752000000005],
 [12.943480000000022, 77.74703000000005],
 [12.975230000000067, 77.75238000000007],
 [13.028477106077231, 77.63188968370176],
 [13.02642000000003, 77.62437000000006],
 [13.038700000000063, 77.66192000000007],
 [12.968020000000024, 77.52114000000006],
 [13.014300000000048, 77.6368500000000

In [265]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coordinates = pd.DataFrame(nbrhd_coordinates, columns=['Latitude', 'Longitude'])

In [266]:
# merge the coordinates into the original dataframe
blr_df['Latitude'] = df_coordinates['Latitude']
blr_df['Longitude'] = df_coordinates['Longitude']

In [267]:
# check the neighborhoods and the coordinates
print(blr_df.shape)
blr_df

(72, 3)


,Neighborhood,Latitude,Longitude
0,"Catonment area, Bangalore",12.966180,77.586900
1,Domlur,12.943290,77.656020
2,Indiranagar,12.973940,77.643900
3,Jeevanbheemanagar,12.966010,77.657670
4,Malleswaram,13.006322,77.568416
5,Pete area,12.966180,77.586900
6,Sadashivanagar,13.014830,77.577710
7,Seshadripuram,12.993550,77.579880
8,Shivajinagar,12.987200,77.604010
9,Ulsoor,12.989080,77.627950


In [225]:
# save the DataFrame as CSV file
blr_df.to_csv("blr_df.csv", index=False)

### 4. Create a map of Bangalore with neighborhoods superimposed on top

In [268]:
# get the coordinates of Kuala Lumpur
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Bangalore, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Bangalore, India are 12.9791198, 77.5912997.


In [228]:
# create map of Bangalore using latitude and longitude values
map_blr = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(blr_df['Latitude'], blr_df['Longitude'], blr_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.4).add_to(map_blr)  
    
map_blr

In [229]:
# save the map as HTML file
map_blr.save('map_blr.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [269]:
# define Foursquare Credentials and Version
CLIENT_ID = 'A3GVNHQ5BVZI1NYEIW3Z3Z1GTKHZQFOGSSOQNWEWJZ1SZPHO' # your Foursquare ID
CLIENT_SECRET = 'HPDQHEBLNATGRUTV0FACP4EBT1YJVG014ZPJYVLLDNQELXWC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A3GVNHQ5BVZI1NYEIW3Z3Z1GTKHZQFOGSSOQNWEWJZ1SZPHO
CLIENT_SECRET:HPDQHEBLNATGRUTV0FACP4EBT1YJVG014ZPJYVLLDNQELXWC


**Now, let's get the top 100 venues that are within a radius of 4000 meters.**

In [270]:
radius = 4000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(blr_df['Latitude'], blr_df['Longitude'], blr_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [271]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5418, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Catonment area, Bangalore",12.96618,77.5869,ITC Gardenia,12.967010,77.595618,Hotel
1,"Catonment area, Bangalore",12.96618,77.5869,JW Marriott Hotel Bengaluru,12.972362,77.595051,Hotel
2,"Catonment area, Bangalore",12.96618,77.5869,UB City,12.971709,77.595905,Shopping Mall
3,"Catonment area, Bangalore",12.96618,77.5869,Toscano,12.971980,77.596066,Italian Restaurant
4,"Catonment area, Bangalore",12.96618,77.5869,Café Noir,12.971995,77.596001,French Restaurant


**Let's check how many venues were returned for each neighorhood**

In [272]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Anjanapura,8,8,8,8,8,8
Arekere,98,98,98,98,98,98
Attibele,4,4,4,4,4,4
BTM Layout,100,100,100,100,100,100
Banashankari,100,100,100,100,100,100
Banaswadi,86,86,86,86,86,86
Basavanagudi,100,100,100,100,100,100
Basaveshwaranagar,100,100,100,100,100,100
Begur,86,86,86,86,86,86


**Let's find out how many unique categories can be curated from all the returned venues**

In [273]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 191 uniques categories.


In [274]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:500]

array(['Hotel', 'Shopping Mall', 'Italian Restaurant',
       'French Restaurant', 'Japanese Restaurant', 'Lounge',
       'Asian Restaurant', 'Sushi Restaurant', 'Fried Chicken Joint',
       'Deli / Bodega', 'Park', 'South Indian Restaurant', 'Theater',
       'Indian Restaurant', 'Ice Cream Shop', 'Burger Joint', 'Brewery',
       'Cupcake Shop', 'Furniture / Home Store', 'Breakfast Spot', 'Pub',
       'American Restaurant', 'Cricket Ground', 'Seafood Restaurant',
       'Bed & Breakfast', 'Plaza', 'Arcade', 'Gym / Fitness Center',
       'Bookstore', 'Bakery', 'Toy / Game Store', 'Chinese Restaurant',
       'Botanical Garden', 'Snack Place', 'Racetrack', 'Tea Room',
       'Dessert Shop', 'Mexican Restaurant', 'Cocktail Bar', 'Café',
       'Coffee Shop', 'Afghan Restaurant', 'Art Gallery',
       'Parsi Restaurant', 'Sandwich Place', 'Steakhouse', 'Wine Bar',
       'Golf Course', 'Andhra Restaurant', 'Electronics Store',
       'Vietnamese Restaurant', 'Restaurant', 'Soccer Sta

In [275]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [276]:
# one hot encoding
blr_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
blr_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [blr_onehot.columns[-1]] + list(blr_onehot.columns[:-1])
blr_onehot = blr_onehot[fixed_columns]

print(blr_onehot.shape)
blr_onehot.head()

(5418, 192)


,Neighborhoods,ATM,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Casino,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,History Museum,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Catonment area, Bangalore",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Catonment area, Bangalore",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Catonment area, Bangalore",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Catonment area, Bangalore",0,0,0,0,0,0,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [277]:
blr_grouped = blr_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(blr_grouped.shape)
blr_grouped

(72, 192)


,Neighborhoods,ATM,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Casino,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,History Museum,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Anjanapura,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.375000,0.000000,0.000000,0.000000,0.00,0.125000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00000

In [278]:
len(blr_grouped[kl_grouped["Shopping Mall"] > 0])

32

**Create a new DataFrame for Shopping Malls only**

In [279]:
blr_malls = blr_grouped[["Neighborhoods","Shopping Mall"]]

In [281]:
blr_malls.head()

,Neighborhoods,Shopping Mall
0,Anjanapura,0.000000
1,Arekere,0.020408
2,Attibele,0.000000
3,BTM Layout,0.020000
4,Banashankari,0.010000


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [282]:
# set number of clusters
kclusters = 3

blr_clustering = blr_malls.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [283]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
blr_merged = blr_malls.copy()

# add clustering labels
blr_merged["Cluster Labels"] = kmeans.labels_

In [284]:
blr_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
blr_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Anjanapura,0.000000,0
1,Arekere,0.020408,1
2,Attibele,0.000000,0
3,BTM Layout,0.020000,0
4,Banashankari,0.010000,0


In [285]:
# merge blr_grouped with blr_data to add latitude/longitude for each neighborhood
blr_merged = blr_merged.join(blr_df.set_index("Neighborhood"), on="Neighborhood")

print(blr_merged.shape)
blr_merged.head() # check the last columns!

(72, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,0,12.85811,77.55909
1,Arekere,0.020408,1,12.88568,77.59668
2,Attibele,0.000000,0,12.77971,77.77058
3,BTM Layout,0.020000,0,12.91488,77.61004
4,Banashankari,0.010000,0,12.92231,77.56988


In [286]:
# sort the results by Cluster Labels
print(blr_merged.shape)
blr_merged.sort_values(["Cluster Labels"], inplace=True)
blr_merged

(72, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,0,12.858110,77.559090
32,Jigani,0.000000,0,12.781130,77.642580
33,Kalyan Nagar,0.000000,0,12.968020,77.521140
34,Kamakshipalya,0.000000,0,12.986990,77.524840
70,Yelahanka,0.000000,0,13.099310,77.592590
36,Kengeri,0.000000,0,12.908720,77.487190
37,Koramangala,0.020000,0,12.920040,77.625460
38,Kothnur,0.000000,0,13.064340,77.648550
40,Kumaraswamy Layout,0.000000,0,12.898190,77.559270
42,Madiwala,0.020000,0,12.920520,77.620900


**Finally, let's visualize the resulting clusters**

In [287]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blr_merged['Latitude'], blr_merged['Longitude'], blr_merged['Neighborhood'], blr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [250]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examining the Clusters

#### Cluster 0

In [288]:
blr_merged.loc[blr_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Anjanapura,0.000000,0,12.858110,77.559090
32,Jigani,0.000000,0,12.781130,77.642580
33,Kalyan Nagar,0.000000,0,12.968020,77.521140
34,Kamakshipalya,0.000000,0,12.986990,77.524840
70,Yelahanka,0.000000,0,13.099310,77.592590
36,Kengeri,0.000000,0,12.908720,77.487190
37,Koramangala,0.020000,0,12.920040,77.625460
38,Kothnur,0.000000,0,13.064340,77.648550
40,Kumaraswamy Layout,0.000000,0,12.898190,77.559270
42,Madiwala,0.020000,0,12.920520,77.620900


#### Cluster 1

In [289]:
blr_merged.loc[blr_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
69,Whitefield,0.040000,1,12.975230,77.752380
1,Arekere,0.020408,1,12.885680,77.596680
65,Varthur,0.040000,1,12.943480,77.747030
62,Thavarekere,0.020000,1,12.922450,77.610330
63,Ulsoor,0.040000,1,12.989080,77.627950
61,Shivajinagar,0.020000,1,12.987200,77.604010
28,J. P. Nagar,0.010000,1,12.908310,77.590240
49,Nandini Layout,0.010000,1,13.014810,77.538910
52,Peenya,0.037037,1,13.031850,77.526790
51,Padmanabhanagar,0.000000,1,12.915470,77.553160


#### Cluster 2

In [290]:
blr_merged.loc[blr_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
29,Jalahalli,0.0625,2,13.0545,77.52658


### Observations:
It can be seen that a majority of shopping malls are located within 4km of Jalahalli (Cluster 2) in Bangalore, while the least number of malls exist in Cluster 0. On the other hand, cluster 1 has a moderate number of shopping malls. Areas in Cluster 0 present a great opportunity to construct new malls which will face lesser competition. There is only 1 area which has a higher density of malls in a 4km radius. 

Hence, I would like to recommend property developers to capitalize on these findings to open new shopping malls in neighborhoods in Cluster 0 and tread with caution in Cluster 1 areas. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid the neighborhood in cluster 2 as it already has high concentration of shopping malls.